In [1]:
from reload_model import reload_main
from util import create_parser, set_seed, logger_setup
from explainer_algo import GNNExplainerEdgeFeature, make_attr_explainer, make_common_attr_explainer
from itertools import product

from torch_geometric.explain import Explainer, GNNExplainer, ModelConfig, ExplainerConfig
from torch_geometric.explain.config import MaskType, ModelMode
from torch_geometric.explain.algorithm.utils import clear_masks, set_masks
from torch_geometric.explain import fidelity, fidelity_curve_auc

from fidelity_custom import fidelity_edge_attr

In [2]:
#create parcer
flags = [
            "--data", "Small_HI",
            "--model", "dir_gin",
            "--ego",
            "--unique_name", "directed_9Jun",
            "--tqdm",
            "--ports",
            "--emlps",
        ]
parser = create_parser()
args = parser.parse_args(flags)


In [3]:
parts_dict = reload_main(args)

2025-06-16 21:23:17,275 [INFO ] Random seed set as 1
2025-06-16 21:23:17,277 [INFO ] Retrieving data
2025-06-16 21:23:19,783 [INFO ] Available Edge Features: ['EdgeID', 'from_id', 'to_id', 'Timestamp', 'Amount Sent', 'Sent Currency', 'Amount Received', 'Received Currency', 'Payment Format', 'Is Laundering']
2025-06-16 21:24:02,689 [INFO ] Illicit ratio = 5177 / 5078345 = 0.10%
2025-06-16 21:24:02,691 [INFO ] Number of nodes (holdings doing transcations) = 515088
2025-06-16 21:24:02,691 [INFO ] Number of transactions = 5078345
2025-06-16 21:24:02,692 [INFO ] Edge features being used: ['Timestamp', 'Amount Received', 'Received Currency', 'Payment Format']
2025-06-16 21:24:02,693 [INFO ] Node features being used: ['Feature'] ("Feature" is a placeholder feature of all 1s)
2025-06-16 21:24:03,190 [INFO ] number of days and transactions in the data: 18 days, 5078345 transactions
2025-06-16 21:24:03,340 [INFO ] Calculate split: [[0, 1, 2, 3, 4, 5], [6, 7], [8, 9, 10, 11, 12, 13, 14, 15, 16, 1

wandb: Currently logged in as: kmishik (kmishik-university-of-bath) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


2025-06-16 21:32:27,929 [INFO ] => loading model checkpoint


C:\Users\ishik\OneDrive\OneNote_Uploads\Documents\GitHub\Thesis_Multi_GNN\reload_model.py:130: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f'{data_

2025-06-16 21:32:28,108 [INFO ] => loaded checkpoint (epoch 35)


100%|██████████| 106/106 [00:29<00:00,  3.54it/s]


number of batch is 106


100%|██████████| 118/118 [00:29<00:00,  4.03it/s]


number of batch is 118


In [17]:
# Baseline: treat edge as an object.
explainer_obj = Explainer(model=parts_dict["model"],
                          algorithm=GNNExplainer(
                              epochs       = 200,
                              lr           = 0.01, 
                              num_hops     = 4,  
                              edge_size    = 1.0,        
                              entropy_reg  = 0.1,
                          ),
                          node_mask_type   = "object", 
                          explanation_type = "model", 
                          edge_mask_type   = "object",
                          model_config     = dict(
                              mode         = "binary_classification",
                              task_level   = "edge",
                              return_type  = "raw",
                          )
                         )

In [18]:
explanation_obj = explainer_obj(
    parts_dict["te_batch"].x,
    parts_dict["te_batch"].edge_index,
    edge_attr =parts_dict["te_batch"].edge_attr,
    # target    =parts_dict["te_batch"].y
)
print(f'Generated explanations in {explanation_obj.available_explanations}')


Generated explanations in ['node_mask', 'edge_mask']


In [19]:
fidelity(explainer_obj, explanation_obj)

(0.0006189942359924316, 0.0039237141609191895)

In [20]:
from torch_geometric.explain import unfaithfulness

metric = unfaithfulness(explainer_obj, explanation_obj)
print(metric)

1.0


In [ ]:
# Edge mask on attribute level
algo = GNNExplainerEdgeFeature(epochs=100, lr=1e-2)

In [9]:
mdl_cfg = ModelConfig(
    mode="multiclass_classification", # to accept tensor is this okay?
    task_level="edge",
    return_type="raw",
)

In [10]:
from torch_geometric.explain import Explainer, fidelity
from torch_geometric.explain.config import MaskType, ModelConfig
explainer = Explainer(
    model            = parts_dict["model"],
    algorithm        = algo,                # same instance as before
    explanation_type = "model",
    node_mask_type   = MaskType.object,
    edge_mask_type   = "object",            # guard
    model_config     = mdl_cfg,
)
explainer.algorithm._explainer_config.edge_mask_type = MaskType.attributes

explanation = explainer(
    x=parts_dict["te_batch"].x, edge_index=parts_dict["te_batch"].edge_index,
    edge_attr=parts_dict["te_batch"].edge_attr, )
    #target=parts_dict["te_batch"].y)

# built-in metric works because edge_mask is 1-D
fid_pos, fid_neg = fidelity(explainer, explanation)
print(f'fidelity on a object level for eddge: {fid_pos}, {fid_neg}')

# The full matrix for visualisation
edge_feat = explanation.edge_feat_mask 


fidelity on a object level for eddge: 0.0026239752769470215, 0.005087852478027344


# Fedility with edge attribute

In [11]:
explainer.model

DirGINe(
  (node_emb): Linear(in_features=2, out_features=66, bias=True)
  (edge_emb): Linear(in_features=6, out_features=66, bias=True)
  (convs): ModuleList(
    (0-1): 2 x DirGINEWrapper(GINEConv(nn=Sequential(
      (0): Linear(in_features=66, out_features=66, bias=True)
      (1): ReLU()
      (2): Linear(in_features=66, out_features=66, bias=True)
    )), alpha=0.5)
  )
  (emlps): ModuleList(
    (0-1): 2 x Sequential(
      (0): Linear(in_features=198, out_features=66, bias=True)
      (1): ReLU()
      (2): Linear(in_features=66, out_features=66, bias=True)
    )
  )
  (batch_norms): ModuleList(
    (0-1): 2 x BatchNorm(66, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (mlp): Sequential(
    (0): Linear(198, 50, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.10527690625126304, inplace=False)
    (3): Linear(50, 25, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.10527690625126304, inplace=False)
    (6): Linear(25, 2, bias=True)
  )
)

In [12]:
fidelity_edge_attr(explainer, explanation)

(0.007391571998596191, 0.002808809280395508)